In [1]:
# 0 Housekeeping. Clear variable space
######################################
from IPython import get_ipython  

ipython = get_ipython()
ipython.magic("reset -f")
ipython = get_ipython()
ipython.magic("load_ext autoreload")
ipython.magic("autoreload 2")

In [2]:
# 1 Import Libraries and Set Global Parameters
##############################################
# 1.1 Import Python Libraries
#############################
from datetime import datetime
print("Run Section 1 Import Libraries and Set Global Parameters...")
begin_time = datetime.now()

import os, sys
import pandas as pd
import geopandas as gpd
import pyarrow.parquet as pq

Run Section 1 Import Libraries and Set Global Parameters...


In [3]:
# 1.2 Set Global Parameters
###########################

if os.getlogin() == "WylieTimmerman":
    path_working = r"C:\OD\OneDrive - Foursquare ITP\Projects\WMATA_AVL"
    os.chdir(os.path.join(path_working))
    sys.path.append(r"C:\OD\OneDrive - Foursquare ITP\Projects\WMATA_AVL")
    path_sp = r"C:\Users\WylieTimmerman\Documents\projects_local\wmata_avl_local"
    path_source_data = os.path.join(path_sp,"data","00-raw")
    path_processed_data = os.path.join(path_sp, "data","02-processed")

elif os.getlogin() == "abibeka":
    path_working = r"C:\Users\abibeka\OneDrive - Kittelson & Associates, Inc\Documents\Github\WMATA_AVL"
    os.chdir(os.path.join(path_working))
    sys.path.append(path_working)
    path_source_data = r"C:\Users\abibeka\OneDrive - Kittelson & Associates, Inc\Documents\WMATA-AVL\Data"
    path_processed_data = os.path.join(path_source_data, "ProcessedData")
    
elif os.getlogin() == "E048374":
    # Working Paths
    path_working = r"C:\Users\E048374\OneDrive - WMATA\rawnav_rachel_fork\WMATA_AVL"
    os.chdir(os.path.join(path_working))
    sys.path.append(r"C:\Users\E048374\OneDrive - WMATA\rawnav_rachel_fork\WMATA_AVL")
    path_source_data = r"\\l-600730\RawNavArchive"
    path_sp = r"C:\Users\E048374\Documents\RawNav"
    path_processed_data = os.path.join(path_sp, "data", "02-processed")
    path_segments = os.path.join(path_working, "data", "02-processed")

else:
    raise FileNotFoundError("Define the path_working, path_source_data, gtfs_dir, \
                            ZippedFilesloc, and path_processed_data in a new elif block")

In [4]:
# Globals
# Queue Jump Routes
q_jump_route_list = ['52']
analysis_routes = q_jump_route_list
# EPSG code for WMATA-area work
wmata_crs = 2248
     
# 1.3 Import User-Defined Package
#################################
import wmatarawnav as wr

In [5]:
seg_pattern = pd.read_csv(os.path.join(path_segments,"stop_seq_pattern_5201_by_intersection.csv"),
                         dtype={'route':str, 'PATTERN_ID':str, 'pattern':'int32'})
seg_pattern

,seg_name_id,geoid,stop_id,route,pattern
0,14th_22,0,0,52,1
1,14th_24,0,0,52,1
2,14th_25,7243,19143,52,1


In [6]:
xwalk_seg_pattern_stop = seg_pattern[['route','pattern','seg_name_id','stop_id']].copy()
xwalk_seg_pattern_stop

,route,pattern,seg_name_id,stop_id
0,52,1,14th_22,0
1,52,1,14th_24,0
2,52,1,14th_25,19143


In [7]:
# 2. Decompose Travel Time
##########################

# 2.0 Setup exports
###################
freeflow_list = []
stop_area_decomp_list = []
traveltime_decomp_list = []

path_exports = (
    os.path.join(
        path_processed_data,
        "exports_{}".format(datetime.now().strftime("%Y%m%d"))
    )
)
if not os.path.isdir(path_exports):
    os.mkdir(path_exports)

In [8]:
# 1 segment
seg = xwalk_seg_pattern_stop.loc[:,'seg_name_id'].values[0]
seg

'14th_22'

In [9]:
print('now on {}'.format(seg))
# 2.1. Read-in Data 
###################
# Reduce rawnav data to runs present in the summary file after filtering.

xwalk_seg_pattern_stop_fil = xwalk_seg_pattern_stop.query('seg_name_id == @seg')

seg_routes = list(xwalk_seg_pattern_stop_fil.route.drop_duplicates())

rawnav_dat = (
    wr.read_cleaned_rawnav(
       analysis_routes_ = seg_routes,
       path = os.path.join(path_processed_data, "rawnav_data.parquet")
    )
    .drop(columns=['blank', 'lat_raw', 'long_raw', 'sat_cnt'])
)

segment_summary = (
    pq.read_table(
        source = os.path.join(path_processed_data,"by_intersection","segment_summary.parquet"),
        filters = [['seg_name_id', "=", seg]],
        use_pandas_metadata = True
    )
    .to_pandas()
)

segment_summary_fil = (
    segment_summary
    .query('~(flag_too_far_any\
              | flag_wrong_order_any\
              | flag_too_long_odom\
              | flag_secs_total_mismatch\
              | flag_odom_total_mismatch)'
    )
)

now on 14th_22


## methodology for segment without a stop

In [10]:
freeflow_seg = wr.decompose_segment_ff(rawnav_dat, segment_summary_fil)
freeflow_seg

,fps_next3,mph
0.01,1.540909,1.050381
0.05,2.121324,1.446028
0.10,7.800000,5.316973
0.15,12.875000,8.776414
0.25,17.000000,11.588275
0.50,25.333333,17.268803
0.75,31.000000,21.131561
0.85,34.500000,23.517382
0.90,37.333333,25.448762
0.95,40.833333,27.834583


In [11]:
segment_ff_val = (
    freeflow_seg
    .loc[0.95]
    .loc["fps_next3"]
)
segment_ff_val

40.83333333333333

In [14]:
segment_runtime_val = (
    freeflow_seg
    .loc[0.50]
    .loc["fps_next3"]
)
segment_runtime_val

25.333333333333332

In [23]:
segments = (
    gpd.read_file(os.path.join(path_segments,"seg_5201_by_intersection.geojson"), dtype={'pattern':'int32'})
    .to_crs(wmata_crs)
)[['seg_name_id', 'name_str', 'geoid', 'stop_id',
       'length', 'geometry']]

In [28]:
segment_length_ft = segments.query('seg_name_id == @seg')['length'].values[0]

In [30]:
segment_ff_sec = segment_length_ft/segment_ff_val
segment_runtime_sec = segment_length_ft/segment_runtime_val
(segment_ff_sec, segment_runtime_sec)

(6.6736552209937665, 10.756878481207057)